In [1]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install oauth2client

     |████████████████████████████████| 102kB 3.1MB/s ta 0:00:011
     |████████████████████████████████| 102kB 16.9MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

In [2]:
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
]

In [3]:
GOOGLE_KEY_FILE = 'gsheets_key.json'

In [4]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials)

In [5]:
wokbook_key = '1mGcKJNUJJvVbN4ycStlQM6PIyWe7ZX1TSSu0OEjH6eY'
workbook = gc.open_by_key(wokbook_key)

In [6]:
sheet = workbook.get_worksheet(2)
values = sheet.get_all_values()

In [7]:
covid_data = pd.DataFrame(values[1:],columns=values[0])

In [8]:
covid_data.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200405,D_20200406,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414
0,Adams County,18001,,,,,1,1,1,1,...,2,2,2,2,2,4,4,5,5,6
1,Allen County,18003,,,,,,,,,...,63,75,87,95,105,119,136,147,156,167
2,Bartholomew County,18005,,,,,,,,,...,54,54,55,66,83,83,108,110,110,110
3,Benton County,18007,,,,,,,,,...,0,0,0,0,0,0,0,2,2,2
4,Blackford County,18009,,,,,,,,,...,3,4,4,4,4,4,5,5,5,6


In [9]:
decatur_johnson_vanderburgh_covid = covid_data.loc[covid_data['County'].isin(['Decatur County', 'Johnson County', 'Vanderburgh County'])]

In [10]:
decatur_johnson_vanderburgh_covid.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200405,D_20200406,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414
15,Decatur County,18031,,,,,,,,,...,80,90,90,99,107,118,123,127,129,153
40,Johnson County,18081,,,,,,3,3,3,...,167,195,212,230,248,270,287,317,327,329
81,Vanderburgh County,18163,,,,,,,,,...,37,40,41,41,49,53,53,56,57,60


In [11]:
decatur_johnson_vanderburgh_covid.columns

Index(['County', 'GEOGID', 'd_20200306', 'd_20200307', 'd_20200308',
       'd_20200309', 'd_20200310', 'd_20200311', 'd_20200312', 'd_20200313',
       'd_20200314', 'd_20200315', 'd_20200316', 'd_20200317', 'd_20200318',
       'd_20200319', 'd_20200320', 'd_20200321', 'd_20200322', 'D_20200323',
       'D_20200324', 'D_20200325', 'D_20200326', 'D_20200327', 'D_20200328',
       'D_20200329', 'D_20200330', 'D_20200331', 'D_20200401', 'D_20200402',
       'D_20200403', 'D_20200404', 'D_20200405', 'D_20200406', 'D_20200407',
       'D_20200408', 'D_20200409', 'D_20200410', 'D_20200411', 'D_20200412',
       'D_20200413', 'D_20200414'],
      dtype='object')

In [12]:
djv_melt = decatur_johnson_vanderburgh_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")

In [13]:
djv_melt['Date'] = djv_melt['Date'].map(lambda x: x.lstrip('dD_'))

In [14]:
djv_melt.head()

,County,GEOGID,Date,Count
0,Decatur County,18031,20200306,
1,Johnson County,18081,20200306,
2,Vanderburgh County,18163,20200306,
3,Decatur County,18031,20200307,
4,Johnson County,18081,20200307,


In [15]:
import numpy as np
vanderburgh_covid = djv_melt.loc[djv_melt['County'].isin(['Vanderburgh County'])]
vanderburgh_covid = vanderburgh_covid.reset_index(drop=True) 
vanderburgh_covid.replace('', np.nan, inplace=True)
vanderburgh_covid.fillna(0, inplace=True)
vanderburgh_covid['Count'] = vanderburgh_covid['Count'].astype(int)

In [16]:
import pickle
vanderburgh_covid.to_pickle("vanderburgh_covid.pkl")

In [17]:
vanderburgh_covid.to_json('static/data/vanderburgh_covid.json')

In [18]:
decatur_covid = djv_melt.loc[djv_melt['County'].isin(['Decatur County'])]
decatur_covid = decatur_covid.reset_index(drop=True) 
decatur_covid.replace('', np.nan, inplace=True)
decatur_covid.fillna(0, inplace=True)
decatur_covid['Count'] = decatur_covid['Count'].astype(int)

In [19]:
decatur_covid.to_pickle("decatur_covid.pkl")

In [20]:
decatur_covid.to_json('static/data/decatur_covid.json')

In [21]:
johnson_covid = djv_melt.loc[djv_melt['County'].isin(['Johnson County'])]
johnson_covid = johnson_covid.reset_index(drop=True) 
johnson_covid.replace('', np.nan, inplace=True)
johnson_covid.fillna(0, inplace=True)
johnson_covid['Count'] = johnson_covid['Count'].astype(int)

In [22]:
johnson_covid.to_pickle("johnson_covid.pkl")

In [23]:
johnson_covid.to_json('static/data/johnson_covid.json')